In [2]:
pip install opencv-python easyocr

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [73]:
import cv2

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    # Use adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Dilate the image to connect individual elements
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=1)
    
    return dilate

In [8]:
import easyocr

text_lists = []
update_text = []
json_text = [] 

def extract_text_from_image(image_path, languages=['en', 'ko']):
    global update_text

    # OCR 리더 초기화
    reader = easyocr.Reader(languages)

    blocked_characters = 'COUBJu니<{}>|]\:[_+=`!@~#$%^&*)(?;\"\''
    # 이미지 파일에서 텍스트 추출
    # 황금비율  width_ths = 15, ycenter_ths = 0.5
    result = reader.readtext(image_path, width_ths = 15, ycenter_ths = 0.5, blocklist=blocked_characters)

    flag = False
    list = []

    # 추출된 텍스트 출력
    for detection in result:
        print(detection[1])

    for (bbox, text, prob) in result:
            list.append(text)
    
    for text in list:
        if flag and ("판매소계" in text or "판매" in text or "판매량" in text or "만" in text or "판" in text or "민" in text or "소계" in text):
            break
        if flag:
            text_lists.append(text)
        if "금액" in text or "수량" in text or "단가" in text:
            flag = True
        

    # 텍스트를 추출할 때,  width_ths = 15, ycenter_ths = 0.5 의 비율로 박스를 조정하면 [품명, 단가 수량 금액] 별로 나온다.
    # 품명은 문제 없지만, 단가 수량 금액을 따로 딕셔너리로 바꾸려면 공백을 기준으로 split을 해야함.
    # 그래서 다음 코드는 짝수번째에 해당하는 배열은 그대로 두고 홀수번째에 해당하는 배열을 조정하는 것이다.
    dt = {}
    flag2 = False

    if("," in text_lists[0] or "." in text_lists[0]) :
        flag2 = True

    for i in range(0, len(text_lists)):
        if(flag2) :
            result_text = text_lists[i].split()
            if(len(result_text) == 4) :
                dt['품목'] = result_text[0]
                dt['단가'] = result_text[1].replace("I", "1")
                dt['수량'] = result_text[2]
                dt['금액'] = result_text[3] 
                json_text.append(dt)
                dt = {}
            else :
                k = 0
                n = 0
                q = 0
                for (j,c) in enumerate(text_lists[i]) :
                    if(c == " " and text_lists[i][j+1] == " ") :
                        dt['품목'] = text_lists[i][0:j]
                        k = j
                        break
                text2 = text_lists[i][k+1:].strip()
                for (m,c) in enumerate(text2) :
                     if(c == " ") :
                         dt['단가'] = text2[0:m]
                         n = m
                         break
                text3 = text2[n+1:].strip()
                for (z,c) in enumerate(text3) :
                    if(c == " ") :
                        dt["수량"] = text3[0:z]
                        q = z
                        break
                        
                text4 = text3[q+1:].strip()
                for (y,c) in enumerate(text4) :
                    if(c == " ") :
                        text4 = text4[0:y] + "." + text4[y+1:]

                dt["금액"] = text4
                
                json_text.append(dt)
                dt={}
        else:
            if i % 2 == 1:
                result_text = text_lists[i].split()
                result_text = [item for item in result_text if item]
                update_text.append(result_text)
                dt['단가'] = result_text[0]
                dt['수량'] = result_text[1]
                if len(result_text) == 3 :
                    dt['금액'] = result_text[2]
                json_text.append(dt)
                dt = {}
            else:
                update_text.append(text_lists[i])
                dt['품목'] = text_lists[i] 
                


# 아쉬운 리스트 : 05(짝수번째 배열을 통째로 인식함), 06(짝수번째 배열을 통째로 인식함), 08(짝수번째 배열을 통째로 인식함) 10(컬럼이 표시 안됨), 12(인식을 못함),
# 13(인식을 못함), 14(인식을 못함), 16(인식을 못함), 18(인식을 못함)
# 수정 성공 : 06(블록리스트를 이용하여 수정함)

# 되는거 : 01, 02, 03, 04, 05, 06, 07, 08, 09, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26
# 깔끔하진 않지만 컬럼에 들어오는거 : 06, 10, 12, 19
# 안되는거 : 
# 이미지 파일 경로 설정
image_path = '/Users/402-07/YoungMan_projectDE/receipt_list/19.jpg'  # 본인이 사용하는 이미지 파일로 경로를 변경하세요.

# 언어 설정 및 이미지에서 텍스트 추출
extract_text_from_image(image_path, languages=['en', 'ko'])
# print(text_lists)
# print(update_text)
print(json_text)


곧죽  공굽받논자들
crs 귀하
2522-016-230 목꼭 00.56.26
Safl     신스j
Diitilg FyingHia   4   A-452
매개션 선   02-2231-0307
해삼논 오 0l0-658E-8181
키북주분ID .   5ats4l5
22-05-231독사    54,000   영수
물명   단가   수량   금액
캠거루다 / 검
27,,0i    27,000
경거두S / 버i
27,0D0 1   27,000
딴대 소계 --   z   2   54,000
소
정일
잔로계   320,o00
합계하    54,000
현금입금하  54,000
당   잔   320,000
곳무리논행 1002-832-59-3330 허색심스흘
부부기 세별도 , 선입금시 전자발행 I내다
kakastory ID 오 Samis45laaa
신상마켓    생스
15223148224741
[{'품목': '캠거루다 / 검', '단가': '27,,0i', '수량': '27,000'}, {'품목': '경거두S / 버i', '단가': '27,0D0', '수량': '1', '금액': '27,000'}]


In [ ]:
import matplotlib.pyplot as plt



In [34]:
import easyocr
import re

text_list = []
def find_by_date(image_path, languages=['en', 'ko']):
    global text_list

    # OCR 리더 초기화
    reader = easyocr.Reader(languages)

    blocked_characters = ';이터,'
    
    # 이미지 파일에서 텍스트 추출
    result = reader.readtext(image_path, width_ths = 0.5, ycenter_ths = 0.5, blocklist=blocked_characters)

    # 추출된 텍스트 확인 
    # for detection in result:
    #     print(detection[1])

    for (bbox, text, prob) in result:
        text_list.append(text)

    # 정규식 패턴
    pattern = r'(\d{4}-\d{2}-\d{2})'
    date_pattern = r'(\d{2}-\d{2})'

    for text in text_list:
        match = re.search(pattern, text)
        month_match = re.search(date_pattern, text)

        if match:
            date_part = match.group(1)  # 날짜 부분
            print(date_part)
            break
        # elif month_match:
        #     month_part = month_match.group(1)
        #     print(month_part)
        #     break

# 사용 가능 01, 02, 03, 04, 05, 07, 08, 09, 11, 15, 17

image_path = '/Users/402-07/YoungMan_projectDE/receipt_list/06.jpg'
print(text_list)
find_by_date(image_path, languages=['en', 'ko'])

[]


In [9]:
import json

json_text = json.dumps(json_text, ensure_ascii=False, indent=4)


print(json_text)


[
    {
        "품목": "캠거루다 / 검",
        "단가": "27,,0i",
        "수량": "27,000"
    },
    {
        "품목": "경거두S / 버i",
        "단가": "27,0D0",
        "수량": "1",
        "금액": "27,000"
    }
]
